In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import sklearn.svm
import cv2
from matplotlib import pyplot as plt
import numpy as np
from os.path import exists
import pandas as pd
import PIL
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import albumentations
import matplotlib.pyplot as plt 
import tensorflow as tf 
import cv2 as cv 
from PIL import Image
import seaborn as sns
import plotly.express as px
import string
import re
from sklearn.metrics.pairwise import cosine_similarity
from skimage.io import imread
torch.cuda.empty_cache()
#cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
#accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

#print("Accelerator type = ",accelerator)
#print("Pytorch verision: ", torch.__version__)

In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ucf101_pth = '/content/drive/My Drive/UCF101'

In [ ]:
#%cd '/content/drive/My Drive/UCF101'

/content/drive/My Drive/UCF101_2


In [ ]:
#!wget https://www.crcv.ucf.edu/data/UCF101/UCF101.rar --no-check-certificate

In [ ]:
#!unrar e UCF101.rar

In [ ]:
# For UCF101 data extraction refer https://github.com/harvitronix/five-video-classification-methods/tree/master/data

In [ ]:
import os
import os.path
import shutil

def get_train_test_lists(version='01'):
    """
    Using one of the train/test files (01, 02, or 03), get the filename
    breakdowns we'll later use to move everything.
    """
    # Get our files based on version.
    test_file = '../ucfTrainTestlist/testlist' + version + '.txt'
    train_file = '../ucfTrainTestlist/trainlist' + version + '.txt'

    # Build the test list.
    with open(test_file) as fin:
        test_list = [row.strip() for row in list(fin)]

    # Build the train list. Extra step to remove the class index.
    with open(train_file) as fin:
        train_list = [row.strip() for row in list(fin)]
        train_list = [row.split(' ')[0] for row in train_list]

    # Set the groups in a dictionary.
    file_groups = {
        'train': train_list,
        'test': test_list
    }

    return file_groups

def move_files(file_groups):
    """This assumes all of our files are currently in _this_ directory.
    So move them to the appropriate spot. Only needs to happen once.
    """
    # Do each of our groups.
    for group, videos in file_groups.items():

        # Do each of our videos.
        for video in videos:

            # Get the parts.
            parts = video.split('/')
            classname = parts[0]
            filename = parts[1]

            if not os.path.exists(video):
              print(" %s Does not exist. Skipping." % (video))
              continue

            # Check if this class exists.
            if not os.path.exists(group + '/' + classname):
                print("Creating folder for %s/%s" % (group, classname))
                os.makedirs(group + '/' + classname)


            # Check if we have already moved this file, or at least that it
            # exists to move.
            if os.path.exists(group+'/'+video):
              print("Already %s exists. Skipping." % (group+'/'+video))
              continue


            # Move it.
            dest = group + '/' + video
            print("Moving %s to %s" % (filename, dest))
            shutil.copy(video,dest)
            #os.rename(filename, dest)

    print("Done.")

def main():
    """
    Go through each of our train/test text files and move the videos
    to the right place.
    """
    # Get the videos in groups so we can move them.
    group_lists = get_train_test_lists()

    # Move the files.
    move_files(group_lists)

In [ ]:
%cd '/content/drive/My Drive/UCF101/'

/content/drive/My Drive/UCF101


In [ ]:
#mkdir train && mkdir test && mkdir sequences && mkdir checkpoints

In [ ]:
#group_lists = get_train_test_lists()

In [ ]:
#print(group_lists['train'])

['ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c01.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c02.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c03.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c04.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c05.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c06.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c07.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c01.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c02.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c03.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c04.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c05.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c01.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c02.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c03.avi', 'ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c04.avi', 'ApplyEyeMak

In [ ]:
'''
group = 'train'
vid = 'ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi'
parts = vid.split('/')
classname = parts[0]
filename = parts[1]
#os.path.exists(group + '/' + classname)
print(group+'/'+vid)
os.path.exists(vid)
'''

train/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi


True

In [ ]:
from tqdm import tqdm, tqdm_notebook
from functools import partial

tqdm = partial(tqdm, position=0, leave=True)

In [ ]:
#tqdm(move_files(group_lists))

In [ ]:
import csv
import glob
import os
import os.path
from subprocess import call

def extract_files():
    """After we have all of our videos split between train and test, and
    all nested within folders representing their classes, we need to
    make a data file that we can reference when training our RNN(s).
    This will let us keep track of image sequences and other parts
    of the training process.
    We'll first need to extract images from each of the videos. We'll
    need to record the following data in the file:
    [train|test], class, filename, nb frames
    Extracting can be done with ffmpeg:
    `ffmpeg -i video.mpg image-%04d.jpg`
    """
    data_file = []
    #folders = ['./train/', './test/']
    folders = ['./train/']


    for folder in folders:
        class_folders = glob.glob(folder + '*')

        for vid_class in class_folders:
            class_files = glob.glob(vid_class + '/*.avi')

            for video_path in class_files:
                # Get the parts of the file.
                video_parts = get_video_parts(video_path)

                train_or_test, classname, filename_no_ext, filename = video_parts

                # Only extract if we haven't done it yet. Otherwise, just get
                # the info.
                if not check_already_extracted(video_parts):
                    # Now extract it.
                    src = train_or_test + '/' + classname + '/' + \
                        filename
                    dest = train_or_test + '/' + classname + '/' + \
                        filename_no_ext + '-%04d.jpg'
                    call(["ffmpeg", "-i", src, dest])

                # Now get how many frames it is.
                nb_frames = get_nb_frames_for_video(video_parts)

                data_file.append([train_or_test, classname, filename_no_ext, nb_frames])

                print("Generated %d frames for %s" % (nb_frames, filename_no_ext))

    with open('data_file.csv', 'w') as fout:
        writer = csv.writer(fout)
        writer.writerows(data_file)

    print("Extracted and wrote %d video files." % (len(data_file)))

def get_nb_frames_for_video(video_parts):
    """Given video parts of an (assumed) already extracted video, return
    the number of frames that were extracted."""
    train_or_test, classname, filename_no_ext, _ = video_parts
    generated_files = glob.glob(train_or_test + '/' + classname + '/' +
                                filename_no_ext + '*.jpg')
    return len(generated_files)

def get_video_parts(video_path):
    """Given a full path to a video, return its parts."""
    parts = video_path.split('/')
    filename = parts[3]
    filename_no_ext = filename.split('.')[0]
    classname = parts[2]
    train_or_test = parts[1]

    return train_or_test, classname, filename_no_ext, filename

def check_already_extracted(video_parts):
    """Check to see if we created the -0001 frame of this file."""
    train_or_test, classname, filename_no_ext, _ = video_parts
    return bool(os.path.exists(train_or_test + '/' + classname +
                               '/' + filename_no_ext + '-0001.jpg'))

def main():
    """
    Extract images from videos and build a new file that we
    can use as our data input file. It can have format:
    [train|test], class, filename, nb frames
    """
    extract_files()

In [ ]:
os.path.exists('./train/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi')

True

In [ ]:
#extract_files()

In [ ]:
!pip install poutyne

     |████████████████████████████████| 143kB 5.0MB/s 


In [ ]:
pwd

'/content/drive/My Drive/UCF101'

In [ ]:
import math
import os
import tarfile
import urllib.request
from shutil import copyfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils import model_zoo
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

from poutyne import set_seeds, Model, ModelCheckpoint, CSVLogger, Experiment

In [ ]:
base_path = '/content/drive/My Drive/UCF101'
train_path = os.path.join(base_path, 'train/')
test_path = os.path.join(base_path, 'test/')

In [ ]:
print(train_path)

/content/drive/My Drive/UCF101/train/


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from skimage import io, transform,data
from torchvision import transforms, utils
import numpy as np
import math
from matplotlib.patches import Ellipse
from skimage.draw import ellipse
import glob
import random
import torchvision.transforms.functional as TF

class ShopeeData(Dataset):
  def __init__(self, root_dir = '/content/drive/My Drive/UCF101',train=True,test=False, transform=None,transform_test=None):
    self.root_dir = root_dir
    self.root_dir_train = self.root_dir + '/train/'
    self.root_dir_test = self.root_dir + '/test/'
    self.train = train
    self.transform = transform
    self.transform_test = transform_test
    self.all_imgs_train = os.listdir(self.root_dir_train)
    self.all_imgs_test = os.listdir(self.root_dir_test)

    self.test = test

  def __len__(self):
    if self.train==True:

      return len(self.all_imgs_train)

    else:

      return len(self.all_imgs_test)   


  def __getitem__(self,index):
    #index+=1
    #index = index%400
    if torch.is_tensor(index):
      index = index.tolist()
    #print(index)

    if self.train==True:
      try:
        img_loc = os.path.join(self.root_dir_train, self.all_imgs_train[index])
        image = Image.open(img_loc).convert("RGB")
      except FileNotFoundError:
        pass
    else:
      try:
        img_loc = os.path.join(self.root_dir_test, self.all_imgs_test[index])
        image = Image.open(img_loc).convert("RGB")
      except FileNotFoundError:
        pass

    #print(img_name)
    #print(ok)
    #image = io.imread(img_name,as_gray=False)
    #print(image.shape)

    #print('Index',index)
    if self.train==False:
      if self.transform_test:
        new_image = self.transform(image)

      sample = {'image': new_image, 'target': new_image}

      return sample


    if self.transform:
      new_image = self.transform(image)

    img_name = img_loc.split('/')[-1]
    posting_id = train_data['posting_id'][train_data['image']==f'{img_name}'].tolist()[0]

    sample = {'image': new_image, 'target': new_image,'name':img_name,'posting_id':posting_id}



    #print(sample['label'])
    #plt.imshow(sample['image'])
    #plt.show()

    return sample

In [ ]:
#Pre processing the data
normalize = transforms.Normalize(mean = [0.485,0.456,0.406],
                                std = [0.229,0.224,0.225])
resize = transforms.Resize((224,224))
centrecrop = transforms.CenterCrop((224,224))
#preprocessor = transforms.Compose(
#    [transforms.ToTensor(),resize, transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),transforms.RandomRotation(45),transforms.RandomPerspective(), transforms.RandomAffine(degrees=20),normalize])


#minority_preprocessor = transforms.Compose([ transforms.ToTensor(),resize, transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),transforms.RandomRotation(45),transforms.RandomPerspective(), transforms.RandomAffine(degrees=20),normalize
#                                   ])


#preprocessor_test = transforms.Compose([ transforms.ToTensor(),resize
#                                   ])

preprocessor_train = transforms.Compose([ transforms.ToTensor(),resize,normalize
                                   ])


train_dataset_full = ShopeeData(root_dir = '/content/drive/My Drive/UCF101',train=True,test=False, transform=preprocessor_train)
test_dataset_full = ShopeeData(root_dir = '/content/drive/My Drive/UCF101',train=False,test=True, transform_test=preprocessor_train)


#trainloader = DataLoader(train_dataset_full, batch_size=2, shuffle=False)
#testloader = DataLoader(test_dataset,batch_size=2, shuffle=False)



In [ ]:
print(len(train_dataset_full))

5


In [ ]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

num_classes = 5
batch_size = 32
learning_rate = 0.001
n_epoch = 30

In [ ]:
norm_coefs = {}
norm_coefs['cub200'] = [(0.47421962,  0.4914721 ,  0.42382449), (0.22846779,  0.22387765,  0.26495799)]
norm_coefs['imagenet'] = [(0.485, 0.456, 0.406), (0.229, 0.224, 0.225)]
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(*norm_coefs['imagenet'])
])
transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(*norm_coefs['imagenet'])
])

train_set = ImageFolder(train_path, transform=transform_train)
test_set = ImageFolder(test_path, transform=transform_test)

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=2)

In [ ]:
print(train_set.class_to_idx)
print(train_set.classes)

{'ApplyEyeMakeup': 0, 'ApplyLipstick': 1, 'Archery': 2, 'BabyCrawling': 3, 'BalanceBeam': 4}
['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam']


In [ ]:
targets = [s[1] for s in train_set.samples]

In [ ]:
print(targets)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import collections
from collections import Counter

In [ ]:
print(dict(Counter(train_set.targets)))

{0: 16751, 1: 13472, 2: 17597, 3: 15517, 4: 9190}


In [ ]:
#model = torchvision.models.video.r3d_18(pretrained=True, progress=True)
model = torchvision.models.resnet50(pretrained=True, progress=True)

In [ ]:
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [ ]:
#print(model)

In [ ]:
def freeze_weights(model):
    for name, param in model.named_parameters():
        if not name.startswith('fc.'):
            param.requires_grad = False

freeze_weights(model)

In [ ]:
optimizer = optim.SGD(model.fc.parameters(), lr=learning_rate, weight_decay=0.001)
loss_function = nn.CrossEntropyLoss()

model_wrap = Model(model, optimizer, loss_function,
              batch_metrics=['accuracy'], epoch_metrics=['f1'],
              device=device)

model_wrap.fit_generator(train_loader, test_loader, epochs=n_epoch)

Epoch:  1/30 Train steps: 2267 Val steps: 943 257.12s loss: 0.461487 acc: 92.627573 fscore_micro: 0.926276 val_loss: 0.412674 val_acc: 86.518381 val_fscore_micro: 0.865184
Epoch:  2/30 Train steps: 2267 Val steps: 943 258.44s loss: 0.172360 acc: 97.974547 fscore_micro: 0.979745 val_loss: 0.372093 val_acc: 85.918388 val_fscore_micro: 0.859184
Epoch:  3/30 Train steps: 2267 Val steps: 943 259.39s loss: 0.120364 acc: 98.702552 fscore_micro: 0.987025 val_loss: 0.355002 val_acc: 85.749329 val_fscore_micro: 0.857493
Epoch:  4/30 Train steps: 2267 Val steps: 943 258.98s loss: 0.096604 acc: 98.909372 fscore_micro: 0.989094 val_loss: 0.341160 val_acc: 86.279710 val_fscore_micro: 0.862797
Epoch:  5/30 Train steps: 2267 Val steps: 943 258.54s loss: 0.080875 acc: 99.117570 fscore_micro: 0.991176 val_loss: 0.348558 val_acc: 85.573640 val_fscore_micro: 0.855736
Epoch:  6/30 Train steps: 2267 Val steps: 943 260.01s loss: 0.070994 acc: 99.201677 fscore_micro: 0.992017 val_loss: 0.349439 val_acc: 85.52

KeyboardInterrupt: ignored

In [ ]:
test_loss, (test_acc, test_f1) = model_wrap.evaluate_generator(test_loader)
print('Test:\n\tLoss: {}\n\tAccuracy: {}\n\tF1-score: {}'.format(test_loss, test_acc, test_f1))

Test steps: 943 75.01s test_loss: 0.335971 test_acc: 86.256505 test_fscore_micro: 0.902954     
Test:
	Loss: 0.33597136154931323
	Accuracy: 86.25650545297842
	F1-score: 0.9029538035392761
